# Instructions on Running this Library

This notebook demonstrates how to operate this library. Currently, only SHAP is fully supported. More features will be added later for SHAP, and ELI5 will also be supported.

In the cell below, sample data and a model are loaded. Note that `classifiers` is a dictionary containing many models, and we are only interested in one of them for the purposes of this demonstration. When running this library yourself, you should be able to substitute your own custom trained classifier (anything compatible with the DeepSHAP library: https://shap.readthedocs.io/en/latest/index.html). Generally, tree-based classifiers from scikit-learn or neural networks are compatible with DeepSHAP, but verify compatibility for your specific model through the link mentioned earlier.

In [1]:
from explainer import SHAPExplainer
import pickle as pkl

data = ["A model to extract geographical locations (e.g. cities) as well as points of interest within those locations.", 
    "Object Detection Model that identifies content areas in purchase order documents",
    "This container translates English to French using Transformers. It can perform batch inference for faster inference.", "Neural machine translation between the writings of Shakespeare and modern English using TensorFlow."]

with open("classifiers.pkl", "rb") as f:
    classifiers = pkl.load(f)

with open("vectorizer.pkl", "rb") as f:
    vectorizer = pkl.load(f)

data_vect = vectorizer.transform(data)
data_vect = data_vect.toarray()


Initiate the `shapExplainer` (custom class) object below, and get the `shap.Explainer` (object from SHAP library) object. The custom `shapExplainer` object requires the following as inputs:

- model
- processed data
- model_type (currently defaults to "general", and should be left at that until more concrete classes are implemented)

In [2]:
shapExplainer = SHAPExplainer(classifiers["image-object-detection"], data_vect, model_type="general")
explanationObject = shapExplainer.getExplanationObject()

To obtain a json of the SHAP bar graph, run the command below. Note that this command has an optional parameter, `max_features`, which limits the maximum number of features to be included in the bar graph json. It also has optional parameters `feature_labels` and `output_labels`.

In [3]:
shapExplainer.getBarPlotJson(explanationObject, output_labels=["Yes", "No"], feature_labels = [f"feature_{i}" for i in range(2407)])

{'Yes': [array([ 647, 1561,  649, 1562,  851]),
  [0.04242852068632599,
   0.022618901694303972,
   0.02085666057286466,
   0.015829319025051344,
   0.012365369947308457,
   0.12109274841906822],
  {647: 'feature_647',
   1561: 'feature_1561',
   649: 'feature_649',
   1562: 'feature_1562',
   851: 'feature_851'}],
 'No': [array([2125, 1867,  520,   73,  603]),
  [0.006060073922393613,
   0.005175630794354047,
   0.004295704255948817,
   0.0032473543904268636,
   0.003173090797867114,
   0.2132396646388534],
  {2125: 'feature_2125',
   1867: 'feature_1867',
   520: 'feature_520',
   73: 'feature_73',
   603: 'feature_603'}]}

The output above is a dictionary with the keys being the index of all output variables. The values are a list. At index 0, there is a numpy array showing the index of the relevant feature, and at index 1 there is a list showing the shap value at the relevant index. Note that, in most cases, the SHAP value list will contain an extra value which is the sum of the SHAP values of all remaining features. The only time this will not happen is if the max number of features is set to greater than the number of features the model uses.

If `feature_labels` was provided in the input, the json list for each output will contain a third item, which will be a mapping from indices to feature name. If `output_labels` was provided, the first-level keys of the returned json will use the output labels rather than an index.